## Mido (Midi Python Library)

In [1]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Function that shows first 15 messages of a midi file 

In [4]:
def midi_head(midi_file=None):
    
    '''
    Each song of our dataset, when converted into midi, contains 2 tracks.
    The second track contains midi event messages that have information about this midi event.
    This function just prints the first 15 messages of the second track of the given song
    
    '''
    
    if midi_file == None:
        print ("Please give valid arguments")
    
    messages = []
    for i, msg in enumerate(midi_file.tracks[1]):
        messages.append(msg)
        
        if i == 15:
            break
            
    return messages

Let's see what these midi messages show

In [5]:
midi_head(song)

[<message program_change channel=0 program=0 time=0>,
 <message control_change channel=0 control=64 value=110 time=0>,
 <message control_change channel=0 control=67 value=117 time=0>,
 <message control_change channel=0 control=64 value=114 time=490>,
 <message control_change channel=0 control=64 value=118 time=94>,
 <message control_change channel=0 control=67 value=121 time=75>,
 <message control_change channel=0 control=64 value=122 time=262>,
 <message note_on channel=0 note=71 velocity=60 time=128>,
 <message control_change channel=0 control=67 value=117 time=37>,
 <message control_change channel=0 control=67 value=113 time=55>,
 <message note_on channel=0 note=71 velocity=0 time=1>,
 <message control_change channel=0 control=64 value=111 time=41>,
 <message control_change channel=0 control=64 value=93 time=19>,
 <message control_change channel=0 control=64 value=76 time=17>,
 <message note_on channel=0 note=55 velocity=44 time=9>,
 <message note_on channel=0 note=71 velocity=54 ti

### Create Function that processes Midi files 

The messages we get from each file, instead of note_on events, also have program_change events. We will not need those in our project, so we create new midi files without them and keeping only the note_on events. You can find and listen to the new midi files in the processed folder of the repo.

We have to be careful though, because program_change events consume time like note_on events do. This must be taken into account when assigning the time attribute of note_on messages in the new midi files.

In [34]:
def processed_Midi(midi_file=None, name=None, path=None):
    
    if midi_file==None or name==None:
        print ("Please give valid arguments")
    
    else:
        new = MidiFile(type=1)   # type=1 just means that the file contains more than one track and all tracks are synchronous, start at the same time.
        new.tracks = [midi_file.tracks[0], MidiTrack()] #Assign the original first track of the midi file passed in the function.
                                                        # This track contains info messages about tempo, key signature etc.
                                                        # The second track is the one we are going to change

        t = 0
        for j, msg in enumerate(midi_file.tracks[1]):
            if msg.dict()['type'] == 'note_on':
                new.tracks[1].append(Message('note_on', note = msg.dict()['note'], velocity = msg.dict()['velocity'], time = t + msg.dict()['time']))
                t = 0

            else:
                t += msg.dict()['time'] # if message type is not note_on, add its time so it is taken into account in the next note_on event
                
        if path == None:
            new.save(name[0:-5]+'_processed.midi')
        else:
            new.save(path+'\\'+name[0:-5]+'_processed.midi')
            
        return new

### Get song names and all midi files in a list

In [35]:
def get_names_and_files(path):
    names = glob.glob(path+'\\*.midi')
    names = [name[len(path)+1:] for name in names]

    files = [MidiFile(path+'\\'+name) for name in names]
    
    return names, files

In [36]:
song_names, song_files = get_names_and_files('dataset\\2004')

In [37]:
for song_file, song_name in zip(song_files, song_names):
    processed_Midi(midi_file=song_file, name=song_name, path='processed')

In [38]:
processed_names, processed_songs = get_names_and_files('processed')

### Play a MidiFile function 

Run the cell below to listen to one of the processed songs we have just created. \n
We will use this function later to play our generated music.

In [ ]:
def play_midi(file):
    mido.set_backend('mido.backends.rtmidi')
    with mido.open_output() as output:
        try:
            print (output)
            with file as midi_file:
                for msg in midi_file.play():
                    print (msg.dict())
                    output.send(msg)
        except KeyboardInterrupt:
            print ()
            output.reset()

play_midi(processed_songs[0])